# Dynamic panel regression (Anderson-Hsiao estimator)

In [ ]:
import numpy as np
import pandas as pd

from linearmodels import PooledOLS          # Pooled model
from linearmodels import RandomEffects      # Random-effect model
from linearmodels import PanelOLS           # Fixed-effect model
from linearmodels import IVGMM              # IV-method

Для датасета `Gasoline` рассмотрим е=регрессию **lgaspcar на lincomep+lrpmg+lcarpcaps**

Спецификация динамической регрессии

$$
	lgaspcar_{it}=\alpha+\gamma lgaspcar_{i,t-1}+\beta_1 lincomep_{it}+\beta_2 lrpmg_{it}+\beta_3 lcarpcaps_{it}+\mu_i+\varepsilon_{it}
$$

Метод оценивания Anderson-Hsiao:

* запишем модель в первых разностях (FD-преобразование) 

$$
	\Delta lgaspcar_{it}=\gamma\Delta lgaspcar_{i,t-1}+\beta_1\Delta lincomep_{it}+\beta_2\Delta lrpmg_{it}+\beta_3\Delta lcarpcaps_{it}+error
$$
* применяем IV-оценки, используя $y_{i,t-2}$ или $\Delta y_{i,t-2}$ как инструменты для $\Delta y_{i,t-1}$

# Оценка 1

In [ ]:
df = pd.read_csv('../datasets/panels-plm/Gasoline.csv')
panel = df.set_index(['country', 'year'])

Используем $y_{i,t-2}$ как инструмент для $\Delta y_{i,t-1}$

Добавим преобразованные переменные в датасет для FD-уравнения

* зависимая переменная $\Delta lgaspcar_{it}$ (`d_lgaspcar`)
* лаг зависимой переменной $\Delta lgaspcar_{i,t-1}$ (`lag_d_lgaspcar`) (эндогенный регрессор)
* экзогенные регрессоры $\Delta lincomep_{it},\Delta lrpmg_{it},\Delta lcarpcaps_{it}$ (`d_lincomep`, `d_lrpmg`, `d_lcarpcaps`)
* инструментальная переменная $lgaspcar_{i,t-2}$ (`lag2_lgaspcar`)

In [ ]:
panel['d_lgaspcar'] = panel['lgaspcar'].groupby(level=0).diff()
panel['lag_d_lgaspcar'] = panel['d_lgaspcar'].groupby(level=0).shift()
panel[['d_lincomep', 'd_lrpmg', 'd_lcarpcap']] = panel[['lincomep', 'lrpmg', 'lcarpcap']].groupby(level=0).diff()
panel['lag2_lgaspcar'] = panel['lgaspcar'].groupby(level=0).shift(periods=2)

Подгоним модель через формулу 

обратим внимание как включаем формулу `lag2_lgaspcar` как инструмент для `lad_d_lgaspcar`

*Замечание* используем `.dropna()` для удаления пропущенных значений (IVGMM не удаляет их самостоятельно!)

In [ ]:
mod_dyn = IVGMM.from_formula(formula='d_lgaspcar~[lag_d_lgaspcar~lag2_lgaspcar]+d_lincomep+d_lrpmg+d_lcarpcap', data=panel.dropna())
res_dyn = mod_dyn.fit()
res_dyn.params.round(3)

In [ ]:
res_dyn.summary

# Оценка 2

In [ ]:
df = pd.read_csv('../datasets/panels-plm/Gasoline.csv')
panel = df.set_index(['country', 'year'])

Используем $\Delta y_{i,t-2}$ как инструмент для $\Delta y_{i,t-1}$

Добавим преобразованные переменные в датасет для FD-уравнения

* зависимая переменная $\Delta lgaspcar_{it}$ (`d_lgaspcar`)
* лаг зависимой переменной $\Delta lgaspcar_{i,t-1}$ (`lad_d_lgaspcar`) (эндогенный регрессор)
* экзогенные регрессоры $\Delta lincomep_{it},\Delta lrpmg_{it},\Delta lcarpcaps_{it}$ (`d_lincomep`, `d_lrpmg`, `d_lcarpcaps`)
* инструментальная переменная $\Delta lgaspcar_{i,t-2}$ (`lad2_d_lgaspcar`)

In [ ]:
panel['d_lgaspcar'] = panel['lgaspcar'].groupby(level=0).diff()
panel['lad_d_lgaspcar'] = panel['d_lgaspcar'].groupby(level=0).shift()
panel[['d_lincomep','d_lrpmg', 'd_lcarpcap']] = panel[['lincomep', 'lrpmg', 'lcarpcap']].groupby(level=0).diff()
panel['lad2_d_lgaspcar'] = panel['d_lgaspcar'].groupby(level=0).shift(periods=2)

Подгоним модель через формулу 

обратим внимание как включаем формулу `lad2_d_lgaspcar` как инструмент для `lad_d_lgaspcar`

*Замечание* используем `.dropna()` для удаления пропущенных значений (IVGMM не удаляет их самостоятельно!)

In [ ]:
mod_dyn = IVGMM.from_formula(formula='d_lgaspcar~[lad_d_lgaspcar~lad2_d_lgaspcar]+d_lincomep+d_lrpmg+d_lcarpcap', data=panel.dropna())
res_dyn = mod_dyn.fit()
res_dyn.params.round(3)

In [ ]:
res_dyn.summary